In [82]:

from langchain.chat_models import ChatOpenAI
from langchain.llms import Ollama
from langchain.agents import Tool , initialize_agent, AgentType
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from tavily import TavilyClient

import os
from dotenv import load_dotenv
load_dotenv()
import smtplib
from email.mime.text import MIMEText
tavily_key = os.getenv("TAVILY_API_KEY")



In [ ]:

##gpt35 = ChatOpenAI(
##    model_name="gpt-3.5-turbo",
 ##   temperature=0.7,
 ##   api_key=os.environ.get("OPENAI_API_KEY"),
##)

llm = Ollama(model="llama3", base_url="http://localhost:11434")

In [84]:
FORMATO_CURSOS = """
1. Curso: Nome do Curso
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
"""

In [85]:

tavily = TavilyClient(api_key="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")

PUBLICAS_KEYWORDS = [
    "uf", "usp", "unicamp", "ufmg", "ufrj", "ufpe", "ufsc", "ufabc", "unb", "if"
]

from duckduckgo_search import DDGS


def func_buscar_cursos(query: str) -> list:
    cursos = []
    query_slug = query.lower().replace(" ", "-")

    with DDGS() as ddgs:
        resultados = ddgs.text(query, max_results=30)  # busca até 30 resultados

        for item in resultados:
            url_lower = item["href"].lower()
            titulo_lower = item["title"].lower()

            # Filtra só universidades públicas
            if any(pub in url_lower or pub in titulo_lower for pub in PUBLICAS_KEYWORDS):
                cursos.append({
                    "titulo": item["title"],
                    "url": item["href"],
                    "conteudo": item["body"][:600] if "body" in item else ""
                })

            if len(cursos) >= 5:  # limite de cursos
                break

    return cursos





buscar_cursos = Tool(
    name="BuscadorCursos",
    func=func_buscar_cursos,
    description="Busca cursos superior na internet ou base de dados publica"
)

In [86]:
# template para estruturar perfil com palavras-chave + cursos


prompt_educacao = ChatPromptTemplate.from_messages([
    ("system", 
        "Você é especialista em educação brasileira. "
        "Sua resposta deve estar estritamente neste formato:"
        f"{FORMATO_CURSOS}\n"
        "- Gere sempre 3 cursos.\n"
        "- Cada curso com 5 universidades listada\n"
        "- **Não inclua nenhuma universidade ou centro privado ou site de conteúdo educacional.**"
        "- Use dados reais, não invente URLs.\n"
        "- Não use JSON nem outro formato de código."
        "- Retorne a resposta em português."
    ),
    ("user", "Perfil: {perfil_usuario}")
])





In [87]:


def enviar_email(resposta: str, destinatario: str):
    """
    Envia a resposta do agente para o e-mail especificado.
    params:
    resposta (str): Conteúdo do e-mail.
    destinatario (str): E-mail do destinatário.
    """
    remetente = "phfdesouza@gmail.com"
    senha = os.environ.get("GMAIL_API_KEY") 

    #criacao da mensagem
    msg = MIMEText(resposta)
    msg['Subject'] = "Resultado análise de Perfil do Agente IA Educação"
    msg['From'] = remetente
    msg['To'] = destinatario

    # envia o email com o conteudo
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(remetente, senha)
        server.send_message(msg)

    print(f"E-mail enviado com sucesso para {destinatario}!")

In [88]:


chain_educacao = LLMChain(
    llm=llm,
    prompt=prompt_educacao,
    verbose=True,
    
)

agente_pesquisador = initialize_agent(
    tools=[buscar_cursos],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,   ##STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION   ZERO_SHOT_REACT_DESCRIPTION  
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3,             # evita loop infinito
    early_stopping_method="generate" ,

)



def orquestrador(perfil_usuario: str):
    sugestoes = chain_educacao.run({"perfil_usuario": perfil_usuario})

    promptpesquisador = (
        f"Com base nestas sugestões: {sugestoes}, "
        "encontre os cursos mais relevantes em instituições e universidades públicas, "
        f"e escreva tudo exatamente assim:\n{FORMATO_CURSOS} listando os 3 cursos e as universidades, - **Não inclua nenhuma universidade ou centro privado ou site de conteúdo educacional.**"
    )

    return agente_pesquisador.run(promptpesquisador)


perfil = "gosto de matemática, fisica e computador. Adoro ficar criando coisas e desmontando e reaproveitando peças velhas para fazer coisas novas"
email = "igor.pulini@ifes.edu.br"


resposta = orquestrador(perfil)
# Verifica se a resposta é multilinha
if len(resposta.splitlines()) > 1:
    print("A resposta é valida, podemos enviar o e-mail.")
    ##enviar_email(resposta, email)
else:
    print("A resposta não é valida, e-mail não enviado.")
print(resposta)










> Entering new LLMChain chain...
Prompt after formatting:
System: Você é especialista em educação brasileira. Sua resposta deve estar estritamente neste formato:
1. Curso: Nome do Curso
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)

- Gere sempre 3 cursos.
- Cada curso com 5 universidades listada
- **Não inclua nenhuma universidade ou centro privado ou site de conteúdo educacional.**- Use dados reais, não invente URLs.
- Não use JSON nem outro formato de código.- Retorne a resposta em português.
Human: Perfil: gosto de matemática, fisica e computador. Adoro ficar criando coisas e desmontando e reaproveitando peças velhas para fazer coisas novas


OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama3`.